### Imports

In [ ]:
import json
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from pyteal import compileTeal, Mode
from contract import approval_program, clear_state_program
from deployment_functions import *
from helper_functions import *

### Create Testing Account

In [ ]:
def generate_algorand_keypair():
    private_key, address = account.generate_account()
    print("My address: {}".format(address))
    print("My private key: {}".format(private_key))
    print("My passphrase: {}".format(mnemonic.from_private_key(private_key)))    
    creator_mnemonic = mnemonic.from_private_key(private_key)

    with open("credentials.json", "r+") as jsonFile:
        data = json.load(jsonFile)

        data['test_address'] = address
        data["test_mnemonic"] = creator_mnemonic

        jsonFile.seek(0)  # bring cursor to the top
        json.dump(data, jsonFile, indent = 6)
        jsonFile.truncate()


### Create Testing asset

In [ ]:
def create_asset_wrapper(private_key):
    asset_id = create_asset(algod_client, private_key)

    with open("credentials.json", "r+") as jsonFile:
        data = json.load(jsonFile)

        data['asset_id'] = asset_id

        jsonFile.seek(0)  # bring cursor to the top
        json.dump(data, jsonFile, indent = 6)
        jsonFile.truncate()
        


### Credentials

In [ ]:
# Opening JSON file
f = open('credentials.json')
 
# returns JSON object as a dictionary
data = json.load(f)
 
creator_address = data["creator_address"]
creator_mnemonic = data["creator_mnemonic"]
creator_private_key = mnemonic.to_private_key(creator_mnemonic)
app_id = data["app_id"]
asset_id = data["asset_id"]
application_address = data["application_address"]
test_address = data["test_address"]
test_mnemonic = data["test_mnemonic"]

# Closing file
f.close()

### Initialize Algod Client

In [ ]:
# user declared algod connection parameters. Node must have EnableDeveloperAPI set to true in its config
algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"

# initialize an algodClient
algod_client = algod.AlgodClient(algod_token, algod_address)

### Compiling smart contract and declaring schema

In [ ]:
# get PyTeal approval program
approval_program_ast = approval_program()
# compile program to TEAL assembly
approval_program_teal = compileTeal(approval_program_ast, mode=Mode.Application, version=6)
# compile program to binary
approval_program_compiled = compile_program(algod_client, approval_program_teal)


# get PyTeal clear state program
clear_state_program_ast = clear_state_program()
# compile program to TEAL assembly
clear_state_program_teal = compileTeal(clear_state_program_ast, mode=Mode.Application, version=6)
# compile program to binary
clear_state_program_compiled = compile_program(algod_client, clear_state_program_teal)


# declare application state storage (immutable)
local_ints = 0
local_bytes = 0
global_ints = 0
global_bytes = 0
global_schema = transaction.StateSchema(global_ints, global_bytes)
local_schema = transaction.StateSchema(local_ints, local_bytes)

### Create the Smart Contract

In [ ]:
def create_app_wrapper():
    # create new application
    app_id = create_app(
        algod_client,
        creator_private_key,
        approval_program_compiled,
        clear_state_program_compiled,
        global_schema,
        local_schema,
    )

    with open("credentials.json", "r+") as jsonFile:
        data = json.load(jsonFile)

        data['app_id'] = app_id

        jsonFile.seek(0)  # bring cursor to the top
        json.dump(data, jsonFile, indent = 6)
        jsonFile.truncate()



### Update the Smart Contract

In [ ]:
def update_app_wrapper():

    application_address = update_app(algod_client, 
                creator_private_key, 
                app_id, 
                approval_program_compiled, 
                clear_state_program_compiled)


    with open("credentials.json", "r+") as jsonFile:
        data = json.load(jsonFile)

        data['application_address'] = application_address

        jsonFile.seek(0)  # bring cursor to the top
        json.dump(data, jsonFile, indent = 6)
        jsonFile.truncate()

update_app_wrapper()

### Opt user into Smart Contract

In [ ]:
if not is_opted_in_app(algod_client, creator_address, app_id):

    # opt into application from pool account
    result = opt_in_app(algod_client, creator_private_key, app_id)

    # print result confirmation transaction confirmation 
    print_confirmation(result)


### Call the Smart Contract

In [ ]:
# call application without arguments
app_args = [b"deposit", asset_id, 1]

call_app(algod_client, 
        creator_private_key, 
        app_id, 
        app_args,
        rekey_to=application_address,
        foreign_assets=[asset_id])

print("Asset holding for user address: ")
print_asset_holding(algod_client, creator_address, asset_id)


print("Asset holding for smart contract address: ")
print_asset_holding(algod_client, application_address, asset_id)

In [ ]:
def transfer_asset():
    creator_private_key = mnemonic.to_private_key(creator_mnemonic)
    params = algod_client.suggested_params()

    # Use the AssetTransferTxn class to transfer assets and opt-in
    txn = transaction.AssetTransferTxn(
        sender=test_address,
        sp=params,
        receiver=creator_address,
        amt=100_000_000,
        index=asset_id)
        
    stxn = txn.sign(test_user_private_key)
       
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid) 
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))    

    print_asset_holding(algod_client, test_address, asset_id)